In [1]:
# Libraries Needed
import os
import re
import sys
import time
from datetime import datetime
import pandas as pd
import numpy as np
from typing import Any, List, Tuple, Union
import sklearn
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score, precision_score, confusion_matrix
import tensorflow
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, CSVLogger
from tensorflow.keras import regularizers
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten,BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import to_categorical
import pickle
from tqdm import tqdm

In [2]:
# create dataset start
def create_dataset(df):
    X_dataset = []
    img_list = df['Path'].tolist()
    for i in tqdm(range(df.shape[0])):
        
        img = image.load_img(img_list[i])
        img = image.img_to_array(img)
        img = img / 255.
        X_dataset.append(img)

    X = np.array(X_dataset)
  # drop unnecessary columns not used for training
    y = np.array(df.drop(['Index', 'Filename','Path','Emotion'], axis=1))

    return X, y
# create dataset end
def label_binarizer(y_intensity,threshold_val):
    return np.where(y_intensity<threshold_val, 0, 1)
def subset_accuracy(y_true,y_pred):
#     y_true = tensorflow.py_function(label_binarizer,(y_true,0.5), tensorflow.double)
#     y_pred = tensorflow.py_function(label_binarizer,(y_pred,0.5), tensorflow.double)
    return tensorflow.py_function(accuracy_score,(y_true,y_pred),tensorflow.double)

def load_model_data(model_path, opt_path):
    model = load_model(model_path,custom_objects={"subset_accuracy":subset_accuracy})
    return model
# define Assume Negative loss
# def loss_an(y_pred, y_true):
#     bce_loss = tensorflow.keras.losses.BinaryCrossentropy()
#     return bce_loss(y_pred,y_true)

# create model start
def create_model(width=224, height=224):
  # load pretrained model 'VGG16'
    base_model = keras.applications.VGG16(
    include_top=False,
    weights="imagenet",
    input_shape=(width, height, 3))
    base_model.trainable = False
    model = Sequential()
    model.add(base_model)
    model.add(Flatten())
    model.add(Dense(12, activation='sigmoid', name='final_au', kernel_initializer='glorot_normal'))
    return model
# create model end


In [3]:
def loss_an(logits, noisy_labels, P):
    """
    This function calculates the loss matrix of assumed negative labels and 
    the corrected negative loss matrix.
    Args:
        logits(tensor): keras model predictions - tensor
        noisy_labels(numpy.ndarray): label vector - numpy.ndarray
        P (dict): parameter dictionary
        
    Returns:
        loss_matirx, corrected loss matrix
    
    """
    # initialize KerasBinaryCrossentropy loss
    BCE = tensorflow.keras.losses.BinaryCrossentropy(
#     from_logits = True,
    reduction = "none"
    )
    # compute assumed negative loss binary cross entropy matrix
    loss_matrix = BCE(noisy_labels,logits)
#     print(f'first loss matrix.shape from first definition of loss_an: {loss_matrix.shape}')
    
    # change noisy labels to boolean vector
    noisy_labels = (noisy_labels>0.5)
    # compute logical not of noisy labels boolean vector
    noisy_labels = tensorflow.logical_not(noisy_labels)
    # change boolean tensor to binary tensor
    noisy_labels = tensorflow.where(noisy_labels, 1, 0)
    # change noisy labels to float32 dtype
    noisy_labels = tensorflow.cast(noisy_labels, dtype=tensorflow.float32)
    # calculate corrected loss matrix
    corrected_loss_matrix = BCE(noisy_labels,logits)
    return loss_matrix, corrected_loss_matrix

def compute_batch_loss(logits, noisy_labels, P):
    # get batch size
    batch_size = int(logits.shape[0])
    # get num classes in multilabel problem
    num_classes = int(logits.shape[1])
    # get boolean matrix for all the labels equal to
    unobserved_mask = (noisy_labels == 0) # 32 * 12
    # calculate batch loss and corrected loss matrices
    loss_matrix, corrected_loss_matrix = loss_an(logits, noisy_labels, P)
    correction_indices = None

    if P['clean_rate'] == 1: # epoch ==1
        final_loss_matrix = loss_matrix
    # after the first epoch
    else:
        if P['mod_scheme'] == 'LL-Cp':
            k = tensorflow.math.ceil(batch_size * num_classes * P['delta_rel'])
        else:
            k = tensorflow.math.ceil(batch_size * num_classes * (1 - P['clean_rate']))
            
        unobserved_mask = tensorflow.where(unobserved_mask,1,0) # 32 * 12
        
        unobserved_mask = tensorflow.cast(unobserved_mask, dtype = tensorflow.float32)
#         # convert loss matrix to ndarray for multiplication
#         # change k to int32 which is reuqeired
#         k = tensorflow.cast(k, dtype = tensorflow.int32)
#         print(k)

#         print(unobserved_mask)
#         print(loss_matrix)
    
        unobserved_loss = tensorflow.math.multiply( unobserved_mask, loss_matrix.numpy().reshape([batch_size,-1]))#.numpy().reshape([32,-1])
# #         print(unobserved_loss.shape)
        flatten_unobserved_loss = tensorflow.reshape(unobserved_loss, [-1])
# #         print(flatten_unobserved_loss)
        topk = tensorflow.math.top_k(flatten_unobserved_loss, tensorflow.cast(k,dtype = tensorflow.int32))
        topk_lossvalue = topk.values[-1]
        
        correction_indices = tensorflow.where(unobserved_loss > topk_lossvalue)
        
        if P['mod_scheme'] in ['LL-Ct','LL-Cp']:
#             cond = tensorflow.less(unobserved_loss, topk_lossvalue)
#             final_loss_matrix = tensorflow.where(cond, tensorflow.where(cond,unobserved_loss,topk_lossvalue),
#                                                 tensorflow.where(cond, topk_lossvalue, unobserved_loss))
#             print(tensorflow.less(unobserved_loss < topk_lossvalue))
#             print(loss_matrix.numpy().reshape([batch_size,-1]))
            final_loss_matrix = tensorflow.where(unobserved_loss < topk_lossvalue, loss_matrix.numpy().reshape([batch_size,-1]), corrected_loss_matrix.numpy().reshape([batch_size,-1]))
#             print(final_loss_matrix)
        else:
            zero_loss_matrix = tensorflow.zeros_like(loss_matrix)
            final_loss_matrix = tensorflow.where(unobserved_loss < topk_lossvalue, loss_matrix, zero_loss_matrix)
    main_loss = tensorflow.math.reduce_mean(final_loss_matrix)

    return main_loss, correction_indices

In [4]:
# load disfa+ dataset
disfa_df = pd.read_csv('AU_image_data.csv')
disfa_df.head()
sample_df = disfa_df.sample(n=950)
X,y = create_dataset(sample_df)
y = np.where(y>=3, 1, 0)

100%|███████████████████████████████████████████████████████████████████████████████| 950/950 [00:08<00:00, 113.81it/s]


In [15]:
# create model 
# create AU model
model = create_model()
# hyperparameter dictionary
P = {
    "num_epochs": 3,
    "lr" : 1e-3,
    "optimizer" :"adam",
    "num_epochs" : 20,
    "clean_rate": 1,
    "delta_rel": 0.1,
    "mod_scheme": "LL-Cp"
}
# instantiate an optimizer
if P['optimizer'] == 'adam':
    opt = tensorflow.keras.optimizers.Adam(learning_rate=P['lr'])

In [16]:
P['mod_scheme']

'LL-Cp'

In [17]:
opt

In [8]:
# initialize the number of epochs to train for, batch sizem, and initial learning rate

# prepare training dataset

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=1)

In [18]:
batch_size = 40

In [19]:
# calculate the number of batch updates per epoch
numUpdates = int(X_train.shape[0]/batch_size)
# loop over the number of epochs
for epoch in range(0, P['num_epochs']):
    best_accuracy = 0
    # loop over the data in batch size increments
    for i in range(0, numUpdates):
        # determine the start and end slice indexes for the current batch
        start = i * batch_size
        end = start + batch_size
        # take a step
#         print(model.trainable_weights)
        with tensorflow.GradientTape() as tape:
            # make prediction using the model and then calculate the loss
            print(f'\n Epoch {epoch}: make prediction {i}:')
            x = X_train[start:end]
            pred = model(x, training = True)
#             print(pred)
            # calculate the loss
            y = y_train[start:end]
            loss, correction_indices = compute_batch_loss(pred, y, P)
            print(correction_indices)
        # calculate gradients using our tape and then update the model weights
        grads = tape.gradient(loss, model.trainable_variables)
        print(grads)
        opt.apply_gradients(zip(grads, model.trainable_variables))
        if P['mod_scheme'] == "LL-Cp" and correction_indices is not None:
            # change batch values
            y[correction_indices[:,0],correction_indices[:,1]] = 1
    # calculate metrics
    y = tensorflow.cast(y, dtype=tensorflow.int32)
    pred = tensorflow.cast(pred, dtype=tensorflow.int32)
#     print(y)
#     print(pred)
    metrics = subset_accuracy(y, pred)
    print(metrics)
    if best_accuracy < metrics:
        best_accuracy = metrics
        best_accuracy_epoch = epoch
        print(f'saving model weight for best metric {best_accuracy}')
        os.path.join('C:\\Users\\user\\Documents\\MSc Computer\\EmotionDetectionUsingCNN\\MissingLabel',f'model-{epoch:02d}-{best_accuracy:.2f}.hdf5')
#     print(metrics)
    P['clean_rate'] -= P['delta_rel']
#         steps(X_train[start:end], y_train[start:end], P)
        



 Epoch 0: make prediction 0:
None
[<tf.Tensor: shape=(25088, 12), dtype=float32, numpy=
array([[ 4.7467030e-03,  6.8172282e-03,  5.7903938e-03, ...,
         1.3122389e-02,  9.2135360e-03,  1.0020803e-02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-1.4194992e-06,  1.6064603e-03,  1.5711115e-03, ...,
         1.2874155e-03,  2.3118667e-03,  1.8747831e-03],
       ...,
       [ 5.4082461e-04,  8.5003226e-04,  8.5837633e-04, ...,
         1.2254197e-03,  1.1141689e-03,  9.0724666e-04],
       [ 1.4482306e-02,  2.6461679e-02,  2.1326790e-02, ...,
         4.4542298e-02,  3.5432111e-02,  3.5034962e-02],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
         0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32)>, <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([0.02329276, 0.0408919 , 0.03170271, 0.03918242, 0.01262129,
       0.03365639, 0.03725169, 0.05323652, 0.02002733, 0.06

None
[<tf.Tensor: shape=(25088, 12), dtype=float32, numpy=
array([[ 4.00482118e-03,  1.68567694e-05,  2.15407368e-03, ...,
        -6.43587962e-04,  2.54727900e-04, -3.73617018e-04],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.88673876e-04,  8.95114340e-07,  1.34601229e-04, ...,
         2.26732438e-10,  2.95867067e-05,  1.44999825e-08],
       ...,
       [ 1.90605101e-06,  1.22080106e-08,  5.29161844e-06, ...,
         4.17662346e-12,  6.62545801e-07,  4.22371166e-10],
       [ 1.30980182e-02, -1.81474537e-03,  7.85853155e-03, ...,
        -1.38036872e-03, -6.82072248e-04, -2.00001500e-03],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]], dtype=float32)>, <tf.Tensor: shape=(12,), dtype=float32, numpy=
array([ 2.2990843e-02, -4.0366836e-03,  1.3252810e-02, -3.9241123e-03,
       -1.7627815e-03, -4.1647218e-03, -7.3726461

ValueError: No gradients provided for any variable: (['final_au/kernel:0', 'final_au/bias:0'],). Provided `grads_and_vars` is ((None, <tf.Variable 'final_au/kernel:0' shape=(25088, 12) dtype=float32, numpy=
array([[ 8.29929486e-03,  2.05741357e-03,  8.00175767e-04, ...,
        -2.29696985e-02,  1.16526440e-03,  1.22488765e-02],
       [-1.07839964e-02,  7.96775625e-04,  2.03310256e-03, ...,
         7.02012982e-03, -7.24897394e-03, -1.04308203e-02],
       [ 9.64653492e-03, -2.18940880e-02, -6.99603837e-03, ...,
        -3.13332514e-03, -4.43929061e-03, -1.19671514e-02],
       ...,
       [ 1.79589633e-02,  7.91123696e-03,  2.50317459e-03, ...,
        -1.79166673e-05,  7.69816991e-03, -2.15028878e-02],
       [-1.51878444e-03, -1.38948467e-02,  4.95119020e-03, ...,
        -7.77135743e-03, -1.17168725e-02, -1.26415165e-02],
       [ 5.87868458e-03,  9.47025232e-03,  7.10674049e-03, ...,
         5.18451736e-04, -1.01820566e-02,  2.01056362e-03]], dtype=float32)>), (None, <tf.Variabl

In [24]:
correction_indices_np = correction_indices.numpy()

In [25]:
correction_indices_np

array([[ 4,  0],
       [ 4,  1],
       [ 4,  3],
       [ 4,  6],
       [ 4,  7],
       [ 4,  8],
       [ 4,  9],
       [ 4, 11],
       [13,  0],
       [13,  1],
       [13,  3],
       [13,  6],
       [13,  7],
       [13,  9],
       [13, 10],
       [13, 11],
       [17,  0],
       [17,  1],
       [17,  3],
       [17,  6],
       [17,  7],
       [17,  9],
       [17, 10],
       [17, 11],
       [18,  0],
       [18,  1],
       [18,  2],
       [18,  3],
       [18,  4],
       [18,  6],
       [18,  7],
       [18,  8],
       [18,  9],
       [18, 11],
       [19,  1],
       [19,  2],
       [19,  3],
       [19,  4],
       [19,  5],
       [19,  7],
       [19,  8],
       [19, 10],
       [19, 11]], dtype=int64)

In [26]:
rows = correction_indices_np.shape[0]
columns = correction_indices_np.shape[1]

In [29]:
for row in range(0,rows):
    for column in range(0, columns):
        if y[row][column] == 1:
            y[row][column]= 0
        else:
            y[row][column] = 1

        print(correction_indices_np[row][column])

4
0
4
1
4
3
4
6
4
7
4
8
4
9
4
11
13
0
13
1
13
3
13
6
13
7
13
9
13
10
13
11
17
0
17
1
17
3
17
6
17
7
17
9
17
10
17
11
18
0
18
1
18
2
18
3
18
4
18
6
18
7
18
8
18
9
18
11
19
1
19
2
19
3
19
4
19
5
19
7
19
8
19
10
19
11


In [ ]:
np_array2 = np.zeros((2, 3), int)


In [189]:
# initialize array with 0s
x = np.zeros((32,12))

In [190]:
x

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.,

In [186]:
# loop over all the rows
for i in range(len(x)):
    rand_column_index = np.random.randint(0,11)
    x[i][rand_column_index] = 1
        

In [188]:
def partial_dataset(noisy_labels):
    
    for i in range(len(noisy_labels)):
        rand_column_index = np.random.randint(0,11)
        noisy_labels[i][rand_column_index]=1
    return noisy_labels

In [191]:
b = partial_dataset(x)
b

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1.,

In [7]:
np_array2 = np.zeros((2, 3), int)

In [8]:
np_array3 = np.ones((2,3),int)

In [205]:
np_array2[0:2] = np_array3

In [206]:
np_array2

array([[1, 1, 1],
       [1, 1, 1]])

In [ ]:
# define Assume Negative loss
# def loss_an(y_pred, y_true):
#     bce_loss = tensorflow.keras.losses.BinaryCrossentropy()
#     return bce_loss(y_pred,y_true)

In [3]:
bce = tensorflow.keras.losses.BinaryCrossentropy(from_logits=False, reduction=tensorflow.keras.losses.Reduction.NONE)

In [14]:
y_true = np.array([[0, 1], [0, 0]])
y_pred = np.array([[-18.6, 0.51], [2.94, -12.8]])

In [13]:
y_true.shape

(2, 2)

In [15]:
bce(y_true, y_pred)

<tf.Tensor: shape=(2,), dtype=float64, numpy=array([0.33667218, 7.71247424])>